###保存和恢复模型


模型可以在训练期间和训练完成后进行保存。这意味着模型可以从任意中断中恢复，并避免耗费比较长的时间在训练上。保存也意味着您可以共享您的模型，而其他人可以通过您的模型来重新创建工作。在发布研究模型和技术时，大多数机器学习从业者分享：

* 用于创建模型的代码
* 模型训练的权重 (weight) 和参数 (parameters) 。

共享数据有助于其他人了解模型的工作原理，并使用新数据自行尝试。

注意：小心不受信任的代码——Tensorflow 模型是代码。有关详细信息，请参阅 [安全使用Tensorflow](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md)。

### 选项

保存 Tensorflow 的模型有许多方法——具体取决于您使用的 API。本指南使用 [tf.keras](https://tensorflow.google.cn/guide/keras)， 一个高级 API 用于在 Tensorflow 中构建和训练模型。有关其他方法的实现，请参阅 TensorFlow  [保存和恢复](https://tensorflow.google.cn/guide/saved_model)指南或[保存到 eager](https://tensorflow.google.cn/guide/eager#object-based_saving)。

##配置
###安装并导入
安装并导入Tensorflow和依赖项：

In [0]:
!pip install -q pyyaml h5py #需要以HDF5格式保存模型

In [21]:
from __future__ import absolute_import, division, print_function,unicode_literals
!pip install -U keras
try:
  %tensorflow_version 2.x
except Exception:
  pass

import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

     |████████████████████████████████| 378kB 2.7MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5
2.1.0-rc1


###获取示例数据集
要演示如何保存和加载权重，您将使用 MNIST 数据集. 要加快运行速度，请使用前1000个示例：

In [4]:
(train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = test_images[:1000].reshape(-1,28*28)/255.0

11493376/11490434 [==============================] - 0s 0us/step


###定义模型
首先构建一个简单的序列（sequential）模型：

In [5]:
# 定义一个简单的序列模型
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512,activation='relu',input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

#创建一个基本的模型实例
model = create_model()

#显示模型的结构
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


##在训练期间保存模型（以 checkpoints 形式保存）
您可以使用训练好的模型而无需从头开始重新训练，或在您打断的地方开始训练，以防止训练过程没有保存。 tf.keras.callbacks.ModelCheckpoint 允许在训练的过程中和结束时回调保存的模型。

###Checkpoint 回调用法
创建一个只在训练期间保存权重的 tf.keras.callbacks.ModelCheckpoint 回调：

In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#创建一个保存模型权重的回调
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
#使用新的回调训练模型
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback]) #通过回调训练

# 这可能会生成与保存优化程序状态相关的警告。
# 这些警告（以及整个笔记本中的类似警告）是防止过时使用，可以忽略。

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 512/1000 [==============>...............] - ETA: 2s - loss: 1.5496 - accuracy: 0.5527  
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 2s 2ms/sample - loss: 1.1439 - accuracy: 0.6820 - val_loss: 0.7416 - val_accuracy: 0.7620
Epoch 2/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.4376 - accuracy: 0.8761
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 254us/sample - loss: 0.4325 - accuracy: 0.8780 - val_loss: 0.5356 - val_accuracy: 0.8390
Epoch 3/10
 544/1000 [===============>..............] - ETA: 0s - loss: 0.2846 - accuracy: 0.9338
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 226us/sample - loss: 0.2830 - accuracy: 0.9240 - val_loss: 0.4753 - val_accuracy: 0.8470
Epoch 4/10
 544/1000 [===============>..............] - ETA: 0s - loss: 0.2166 - accuracy: 0.9449
Epoch

这将创建一个 TensorFlow checkpoint 文件集合，这些文件在每个 epoch 结束时更新：



In [7]:
!ls {checkpoint_dir}

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


创建一个新的未经训练的模型。仅恢复模型的权重时，必须具有与原始模型具有相同网络结构的模型。由于模型具有相同的结构，您可以共享权重，尽管它是模型的不同实例。 现在重建一个新的未经训练的模型，并在测试集上进行评估。未经训练的模型将在机会水平（chance levels）上执行（准确度约为10％）：

In [8]:
#创建一个基本模型实例
model = create_model()

#评估模型
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 2.4074 - accuracy: 0.0470
Untrained model, accuracy:  4.70%


然后从 checkpoint 加载权重并重新评估：

In [9]:
#加载权重
model.load_weights(checkpoint_path)

#重新评估模型
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print("Restored model,accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4160 - accuracy: 0.8680
Restored model,accuracy: 86.80%


###checkpoint 回调选项
回调提供了几个选项，为 checkpoint 提供唯一名称并调整 checkpoint 频率。

训练一个新模型，每五个 epochs 保存一次唯一命名的 checkpoint ：

In [10]:
# 在文件名中包含 epoch (使用 `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#创建一个回调，每5个epochs保存模型的权重
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)
#创建一个新的模型实例
model = create_model()

#使用`checkpoint_path`格式保存权重
model.save_weights(checkpoint_path.format(epoch=0))

#使用新的回调*训练*模型
model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


现在查看生成的 checkpoint 并选择最新的 checkpoint ：

In [11]:
! ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.data-00001-of-00002
cp-0000.ckpt.data-00000-of-00002  cp-0025.ckpt.index
cp-0000.ckpt.data-00001-of-00002  cp-0030.ckpt.data-00000-of-00002
cp-0000.ckpt.index		  cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00000-of-00002  cp-0030.ckpt.index
cp-0005.ckpt.data-00001-of-00002  cp-0035.ckpt.data-00000-of-00002
cp-0005.ckpt.index		  cp-0035.ckpt.data-00001-of-00002
cp-0010.ckpt.data-00000-of-00002  cp-0035.ckpt.index
cp-0010.ckpt.data-00001-of-00002  cp-0040.ckpt.data-00000-of-00002
cp-0010.ckpt.index		  cp-0040.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00000-of-00002  cp-0040.ckpt.index
cp-0015.ckpt.data-00001-of-00002  cp-0045.ckpt.data-00000-of-00002
cp-0015.ckpt.index		  cp-0045.ckpt.data-00001-of-00002
cp-0020.ckpt.data-00000-of-00002  cp-0045.ckpt.index
cp-0020.ckpt.data-00001-of-00002  cp-0050.ckpt.data-00000-of-00002
cp-0020.ckpt.index		  cp-0050.ckpt.data-00001-of-00002
cp-0025.ckpt.data-00000-of-00002  cp-0050.ckpt.index


In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

注意: 默认的 tensorflow 格式仅保存最近的5个 checkpoint 。

如果要进行测试，请重置模型并加载最新的 checkpoint ：

In [13]:
#创建一个新模型实例
model = create_model()

#加载以前保存的权重
model.load_weights(latest)

#重新评估模型
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print("Restored model,accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4840 - accuracy: 0.8750
Restored model,accuracy: 87.50%


##这些文件是什么？
上述代码将权重存储到 checkpoint—— 格式化文件的集合中，这些文件仅包含二进制格式的训练权重。 Checkpoints 包含：

* 一个或多个包含模型权重的分片。
* 索引文件，指示哪些权重存储在哪个分片中。

如果你只在一台机器上训练一个模型，你将有一个带有后缀的碎片： .data-00000-of-00001

##手动保存权重
您将了解如何将权重加载到模型中。使用 Model.save_weights 方法手动保存它们同样简单。默认情况下， tf.keras 和 save_weights 特别使用 TensorFlow checkpoints 格式 .ckpt 扩展名和 ( 保存在 HDF5 扩展名为 .h5 保存并序列化模型 )：

In [14]:
#保存权重
model.save_weights('./checkpoints/my_checkpoint')

#创建模型实例
model = create_model()

#Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

#Evaluate the model
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4840 - accuracy: 0.8750
Restored model, accuracy: 87.50%


##保存整个模型
模型和优化器可以保存到包含其状态（权重和变量）和模型参数的文件中。这可以让您导出模型，以便在不访问原始 python 代码的情况下使用它。而且您可以通过恢复优化器状态的方式，从中断的位置恢复训练。

保存完整模型会非常有用——您可以在 TensorFlow.js (HDF5, Saved Model) 加载他们，然后在 web 浏览器中训练和运行它们，或者使用 TensorFlow Lite 将它们转换为在移动设备上运行(HDF5, Saved Model)

###将模型保存为HDF5文件
Keras 可以使用 HDF5 标准提供基本保存格式。出于我们的目的，可以将保存的模型视为单个二进制blob：

In [15]:
# 创建一个新的模型实例
model = create_model()

#训练模型
model.fit(train_images,train_labels,epochs=5)

#将整个模型保存为HDF5文件 
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 406us/sample - loss: 1.1711 - accuracy: 0.6850
Epoch 2/5
1000/1000 [==============================] - 0s 98us/sample - loss: 0.4395 - accuracy: 0.8820
Epoch 3/5
1000/1000 [==============================] - 0s 101us/sample - loss: 0.3138 - accuracy: 0.9120
Epoch 4/5
1000/1000 [==============================] - 0s 152us/sample - loss: 0.2357 - accuracy: 0.9390
Epoch 5/5
1000/1000 [==============================] - 0s 106us/sample - loss: 0.1536 - accuracy: 0.9710


现在，从该文件重新创建模型：

In [16]:
# 重新创建完全相同的模型，包括其权重和优化程序
new_model = keras.models.load_model('my_model.h5')

#显示网络结构
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


检查其准确率（accuracy）：

In [17]:
loss,acc = new_model.evaluate(test_images,test_labels,verbose=2)
print("Restored model,accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4184 - accuracy: 0.8600
Restored model,accuracy: 86.00%


这项技术可以保存一切:

* 权重
* 模型配置(结构)
* 优化器配置

Keras 通过检查网络结构来保存模型。目前，它无法保存 Tensorflow 优化器（调用自 tf.train）。使用这些的时候，您需要在加载后重新编译模型，否则您将失去优化器的状态。

###通过 saved_model 保存
注意：这种保存 tf.keras 模型的方法是实验性的，在将来的版本中可能有所改变。

建立一个新模型，然后训练它：

In [18]:
model = create_model()

model.fit(train_images,train_labels,epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 562us/sample - loss: 1.1456 - accuracy: 0.6820
Epoch 2/5
1000/1000 [==============================] - 0s 114us/sample - loss: 0.4250 - accuracy: 0.8750
Epoch 3/5
1000/1000 [==============================] - 0s 100us/sample - loss: 0.2848 - accuracy: 0.9280
Epoch 4/5
1000/1000 [==============================] - 0s 98us/sample - loss: 0.2028 - accuracy: 0.9590
Epoch 5/5
1000/1000 [==============================] - 0s 100us/sample - loss: 0.1541 - accuracy: 0.9700


创建一个 saved_model，并将其放在带有 tf.keras.experimental.export_saved_model 的带时间戳的目录中：

In [23]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

model.save(saved_model_path)
saved_model_path

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./saved_models/1579175140/assets


'./saved_models/1579175140'

列出您保存的模型：

In [24]:
!ls saved_models/

1579175140


从保存的模型重新加载新的 Keras 模型：

In [25]:
new_model = tf.keras.models.load_model(saved_model_path)

#显示网络结构
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


使用恢复的模型运行预测：

In [26]:
model.predict(test_images).shape

(1000, 10)

In [27]:
# 必须在评估之前编译模型。
# 如果仅部署已保存的模型，则不需要此步骤。

new_model.compile(optimizer=model.optimizer,#保留已加载的优化程序
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

#评估已恢复的模型
loss,acc = new_model.evaluate(test_images,test_labels,verbose=2)
print("Restored model,accuracy:{:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4237 - accuracy: 0.8650
Restored model,accuracy:86.50%
